#Connect to drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from openai import OpenAI
api_key = ''

In [ ]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=api_key,
)

In [ ]:
import json
f = open('/content/drive/MyDrive/CSCI544/Project/dataset/dataset/multiarith.json')
data = json.load(f)

In [ ]:
training_data = []
for i in range(int(len(data['data'])*0.7)):
  d = {}
  d['prompt'] = data['data'][i]['question']
  chat_completion = client.chat.completions.create(
  messages=[
        {
            "role": "user",
            "content": data['data'][i]['question']+". Think step by step.",
        }
    ], model="gpt-4",)
  d["completion"] = chat_completion.choices[0].message.content + "--> " + data['data'][i]['answer'] + " END"
  training_data.append(d)

In [ ]:
training_file_name = "training_data.jsonl"

def prepare_data(dictionary_data, final_file_name):
    with open(final_file_name, 'w') as outfile:
        for entry in dictionary_data:
        	json.dump(entry, outfile)
        	outfile.write('\n')

prepare_data(training_data, "training_data.jsonl")

In [ ]:
training_file_id = client.files.create(
  file=open(training_file_name, "rb"),
  purpose="fine-tune"
)
print(f"Training File ID: {training_file_id}")

Training File ID: FileObject(id='file-kcjw9Z2AP6eBtVjwxRiNmk47', bytes=200373, created_at=1711413079, filename='training_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)


In [ ]:
response = client.fine_tuning.jobs.create(
  training_file=training_file_id.id,
  model="babbage-002",
  hyperparameters={
    "n_epochs": 5
  }
)
job_id = response.id
status = response.status

print(f'Fine-tunning model with jobID: {job_id}.')
print(f"Training Response: {response}")
print(f"Training Status: {status}")

In [ ]:
result = client.fine_tuning.jobs.list()
fine_tuned_model = result.data[0].fine_tuned_model
result.data[0]

FineTuningJob(id='ftjob-sOYHdev7H9oPtXPvnC2b03jE', created_at=1711413105, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=30, batch_size=8, learning_rate_multiplier=2), model='babbage-002', object='fine_tuning.job', organization_id='org-JrmVss1Zj1qhFSpn2QGWUSiR', result_files=[], status='cancelled', trained_tokens=None, training_file='file-kcjw9Z2AP6eBtVjwxRiNmk47', validation_file=None, user_provided_suffix=None)

In [ ]:
correct_predictions = 0
for i in range(int(len(data['data'])*0.7), len(data['data'])):

  new_prompt = data['data'][i]['question']
  answer = client.completions.create(
    model=fine_tuned_model,
    prompt=new_prompt,
    max_tokens = 150
  )
  try:
    pred = int(answer.choices[0].text.split("-->")[1].split("END")[0].strip())
  except:
    pred = -9999
    pass
  ans = data['data'][i]['answer']
  if '.' in data['data'][i]['answer']:
    pred=float(pred)
    ans = float(ans)
  else:
    ans = int(ans)
  if pred == ans:
    correct_predictions+= 1

total_data_points = len(data['data']) - int(len(data['data'])*0.7)
print("Accuracy = ",(correct_predictions/total_data_points)*100, "%")